In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AGA/Data/Social_Network_Ads.csv')
dataset = data
dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [ ]:
X = dataset.iloc[:,1:-1].values
Y = dataset.iloc[:,-1].values
X

array([['Male', 19, 19000],
       ['Male', 35, 20000],
       ['Female', 26, 43000],
       ...,
       ['Female', 50, 20000],
       ['Male', 36, 33000],
       ['Female', 49, 36000]], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lcoder = LabelEncoder()
X[:,0] = lcoder.fit_transform(X[:,0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25,random_state=0)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier

classifire = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=1)
classifire.fit(X_train,Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=1)

In [ ]:
# 5) Extract and display importances
importances = classifire.feature_importances_
feat_names = dataset.columns[1:-1]
feat_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)
print("\nFeature importances (Random Forest):")
print(feat_imp)


Feature importances (Random Forest):
Age                0.499448
EstimatedSalary    0.485053
Gender             0.015500
dtype: float64


In [ ]:
# 7) Select top 2 best features automatically

selected_features = feat_imp.index[:2].tolist()

print("Top 2 selected features:", selected_features)


Top 2 selected features: ['Age', 'EstimatedSalary']


In [ ]:
original_feature_names = list(dataset.columns[1:-1])
feature_indices = [original_feature_names.index(f) for f in selected_features]
X_train_sel = X_train[:, feature_indices].copy()
X_test_sel  = X_test[:, feature_indices].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_test_sel  = scaler.transform(X_test_sel)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=42)
lr.fit(X_train_sel, Y_train)

LogisticRegression(random_state=42)

In [ ]:
# 10) Evaluate Logistic Regression on test set
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = lr.predict(X_test_sel)
acc = accuracy_score(Y_test, y_pred)
print("\nLogistic Regression Test Accuracy:", round(acc, 4))
print("\nClassification Report:\n", classification_report(Y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, y_pred))


Logistic Regression Test Accuracy: 0.89

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.96      0.92        68
           1       0.89      0.75      0.81        32

    accuracy                           0.89       100
   macro avg       0.89      0.85      0.87       100
weighted avg       0.89      0.89      0.89       100


Confusion Matrix:
 [[65  3]
 [ 8 24]]


In [38]:
# 1) Prepare X and y exactly as you used before
X = dataset.iloc[:, 1:-1].values   # columns 2..(last-1) -> e.g., Age, EstimatedSalary
y = dataset.iloc[:, -1].values    # last column -> Purchased

In [39]:
from sklearn.preprocessing import LabelEncoder
lcoder = LabelEncoder()
X[:,0] = lcoder.fit_transform(X[:,0])

In [40]:
# Optional: get feature names for mapping indices -> names
feature_names = dataset.columns[1:-1].tolist()
print("Features considered:", feature_names)

Features considered: ['Gender', 'Age', 'EstimatedSalary']


In [41]:
from sklearn.feature_selection import SelectKBest, chi2
selector_uni = SelectKBest(score_func=chi2, k=3)
X_uni = selector_uni.fit_transform(X, y)

selected_uni = feature_names[selector_uni.get_support()]
print("\n[1a] Selected Best Features (Univariate):")
print(selected_uni)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# 2) Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=Y)

In [ ]:
# 3) Apply SelectKBest with ANOVA F-test (f_classif)
from sklearn.feature_selection import SelectKBest, f_classif
k = 2   # choose top-2 features
selector = SelectKBest(score_func=f_classif, k=k)
selector.fit(X_train, y_train)

SelectKBest(k=2)

In [ ]:
# 4) Get scores and p-values (f_classif returns arrays)
scores = selector.scores_      # F-scores for each feature
pvalues = selector.pvalues_    # p-values for each feature

In [ ]:
# Map scores to feature names and show sorted
score_series = pd.Series(scores, index=feature_names).sort_values(ascending=False)
pval_series = pd.Series(pvalues, index=feature_names).loc[score_series.index]

In [ ]:
print("\nUnivariate feature scores (F-stat):")
print(score_series)
print("\nCorresponding p-values:")
print(pval_series)


Univariate feature scores (F-stat):
Age                197.047043
EstimatedSalary     63.632237
Gender               0.306230
dtype: float64

Corresponding p-values:
Age                3.623392e-35
EstimatedSalary    2.736403e-14
Gender             5.803916e-01
dtype: float64


In [ ]:
# 5) Which features were selected?
mask = selector.get_support()  # boolean mask of selected features
selected_feature_names = [name for name, keep in zip(feature_names, mask) if keep]
print("\nSelected top-{} features by univariate selection: {}".format(k, selected_feature_names))


Selected top-2 features by univariate selection: ['Age', 'EstimatedSalary']


In [ ]:
# 6) Prepare data using only selected features, then scale
X_train_sel = selector.transform(X_train)
X_test_sel  = selector.transform(X_test)

In [ ]:
scaler = StandardScaler()
X_train_sel = scaler.fit_transform(X_train_sel)
X_test_sel  = scaler.transform(X_test_sel)

In [ ]:
# 7) Train Logistic Regression on selected features
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_sel, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
y_pred = lr.predict(X_test_sel)
acc = accuracy_score(y_test, y_pred)
print("\nLogistic Regression Test Accuracy (top-{} features): {:.4f}".format(k, acc))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



Logistic Regression Test Accuracy (top-2 features): 0.8375

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.94      0.88        51
           1       0.86      0.66      0.75        29

    accuracy                           0.84        80
   macro avg       0.85      0.80      0.81        80
weighted avg       0.84      0.84      0.83        80


Confusion Matrix:
 [[48  3]
 [10 19]]


In [42]:
# ----------------------------------------------------------
# IMPORT LIBRARIES
# ----------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ----------------------------------------------------------
# LOAD IRIS DATASET
# ----------------------------------------------------------
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print("Original Data Shape:", X.shape)

# ----------------------------------------------------------
# 1(a) UNIVARIATE FEATURE SELECTION (CHI-SQUARE)
# ----------------------------------------------------------
selector_uni = SelectKBest(score_func=chi2, k=3)
X_uni = selector_uni.fit_transform(X, y)

selected_features_uni = np.array(feature_names)[selector_uni.get_support()]
print("\n[1a] Selected 3 Best Features (Univariate):")
print(selected_features_uni)

# Train-test split
X_train_uni, X_test_uni, y_train_uni, y_test_uni = train_test_split(
    X_uni, y, test_size=0.20, random_state=42, stratify=y
)

# Standard scaling (important for logistic regression)
scaler_uni = StandardScaler()
X_train_uni = scaler_uni.fit_transform(X_train_uni)
X_test_uni = scaler_uni.transform(X_test_uni)

# Train Logistic Regression
lr_uni = LogisticRegression(max_iter=1000)
lr_uni.fit(X_train_uni, y_train_uni)

# Evaluate Model
print("\n[1a] Logistic Regression Evaluation (Univariate):")
y_pred_uni = lr_uni.predict(X_test_uni)
print("Accuracy:", accuracy_score(y_test_uni, y_pred_uni))
print(classification_report(y_test_uni, y_pred_uni))
print("Confusion Matrix:\n", confusion_matrix(y_test_uni, y_pred_uni))

# Predict unseen sample
unseen = np.array([[6.121, 2.8121, 4.7121, 1.2121]])
unseen_uni = selector_uni.transform(unseen)     # select same 3 features
unseen_uni = scaler_uni.transform(unseen_uni)   # scale
pred_uni = lr_uni.predict(unseen_uni)

print("\n[1a] Predicted Species for Unseen Sample (Univariate):",
      target_names[pred_uni][0])

# ----------------------------------------------------------
# 1(b) RANDOM FOREST FEATURE SELECTION
# ----------------------------------------------------------
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X, y)

importances = rf.feature_importances_
sorted_idx = np.argsort(importances)[::-1]

# Select top 3 features
top3_idx = sorted_idx[:3]
X_rf = X[:, top3_idx]
selected_features_rf = np.array(feature_names)[top3_idx]

print("\n[1b] Selected 3 Best Features (Random Forest):")
print(selected_features_rf)

# Train-test split
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X_rf, y, test_size=0.20, random_state=42, stratify=y
)

# Standard scaling
scaler_rf = StandardScaler()
X_train_rf = scaler_rf.fit_transform(X_train_rf)
X_test_rf = scaler_rf.transform(X_test_rf)

# Train Logistic Regression
lr_rf = LogisticRegression(max_iter=1000)
lr_rf.fit(X_train_rf, y_train_rf)

# Evaluate model
print("\n[1b] Logistic Regression Evaluation (Random Forest):")
y_pred_rf = lr_rf.predict(X_test_rf)
print("Accuracy:", accuracy_score(y_test_rf, y_pred_rf))
print(classification_report(y_test_rf, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test_rf, y_pred_rf))

# Predict unseen sample
unseen_rf = unseen[:, top3_idx]      # choose same 3 RF-selected features
unseen_rf = scaler_rf.transform(unseen_rf)
pred_rf = lr_rf.predict(unseen_rf)

print("\n[1b] Predicted Species for Unseen Sample (Random Forest):",
      target_names[pred_rf][0])


Original Data Shape: (150, 4)

[1a] Selected 3 Best Features (Univariate):
['sepal length (cm)' 'petal length (cm)' 'petal width (cm)']

[1a] Logistic Regression Evaluation (Univariate):
Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]

[1a] Predicted Species for Unseen Sample (Univariate): versicolor

[1b] Selected 3 Best Features (Random Forest):
['petal length (cm)' 'petal width (cm)' 'sepal length (cm)']

[1b] Logistic Regression Evaluation (Random Forest):
Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

           0       1.00      1.00